## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv", index_col=[0])
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Turbat,PK,26.0023,63.0440,89.58,48,92,5.37,overcast clouds
1,Ushuaia,AR,-54.8000,-68.3000,42.46,49,20,18.41,few clouds
2,Sambava,MG,-14.2667,50.1667,67.82,85,7,8.48,clear sky
3,Kenai,US,60.5544,-151.2583,45.03,57,75,23.02,broken clouds
4,Nouadhibou,MR,20.9310,-17.0347,71.58,88,59,13.80,broken clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip?"))
max_temp=float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [23]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_locations_df=city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & \
                                      (city_data_df["Max Temp"]>=min_temp)]
preferred_locations_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Turbat,PK,26.0023,63.0440,89.58,48,92,5.37,overcast clouds
5,Hilo,US,19.7297,-155.0900,81.28,89,90,5.75,light rain
6,Vaini,TO,-21.2000,-175.2000,75.36,88,75,9.22,light rain
12,Kapaa,US,22.0752,-159.3190,84.18,70,75,4.00,broken clouds
13,Atuona,PF,-9.8000,-139.0333,78.21,72,5,16.91,clear sky
17,Bengkulu,ID,-3.8004,102.2655,82.04,68,96,5.01,light rain
19,Ballina,AU,-28.8667,153.5667,75.38,73,40,9.22,moderate rain
20,San Patricio,US,28.0170,-97.5169,82.26,76,80,10.94,broken clouds
23,Bilma,NE,18.6853,12.9164,82.13,18,0,17.98,clear sky
31,Georgetown,MY,5.4112,100.3354,85.95,81,20,3.00,light rain


In [73]:
# 4a. Determine if there are any empty rows.
preferred_locations_df.count()
bool_series=pd.isnull(preferred_locations_df["Max Temp"])
preferred_locations_df[bool_series]

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description


In [74]:
# 4a. Determine if there are any empty rows... look to see if any series has less valid entries
preferred_locations_df.count()

City           238
Country        238
Lat            238
Lng            238
Max Temp       238
Humidity       238
Cloudiness     238
Wind Speed     238
Description    238
dtype: int64

In [75]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_locations_df.dropna()


In [76]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Turbat,PK,89.58,overcast clouds,26.0023,63.0440,
5,Hilo,US,81.28,light rain,19.7297,-155.0900,
6,Vaini,TO,75.36,light rain,-21.2000,-175.2000,
12,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,
13,Atuona,PF,78.21,clear sky,-9.8000,-139.0333,
17,Bengkulu,ID,82.04,light rain,-3.8004,102.2655,
19,Ballina,AU,75.38,moderate rain,-28.8667,153.5667,
20,San Patricio,US,82.26,broken clouds,28.0170,-97.5169,
23,Bilma,NE,82.13,clear sky,18.6853,12.9164,
31,Georgetown,MY,85.95,light rain,5.4112,100.3354,


In [77]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,"type": "lodging","key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"]=f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    hotels=requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [79]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Turbat,PK,89.58,overcast clouds,26.0023,63.0440,Mubashar Quarters
5,Hilo,US,81.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Vaini,TO,75.36,light rain,-21.2000,-175.2000,Keleti Beach Resort
12,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Atuona,PF,78.21,clear sky,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
705,Hope,US,77.07,few clouds,33.6671,-93.5916,Best Western Hope
708,Hamada,JP,77.72,broken clouds,34.8833,132.0833,Shimane Hamada Washington Hotel Plaza
709,Kandrian,PG,82.60,moderate rain,-6.2167,149.5500,Kandrian Station
713,Darab,IR,78.78,clear sky,28.7519,54.5444,Naghshe shapoor Hotel


In [80]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Turbat,PK,89.58,overcast clouds,26.0023,63.0440,Mubashar Quarters
5,Hilo,US,81.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Vaini,TO,75.36,light rain,-21.2000,-175.2000,Keleti Beach Resort
12,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Atuona,PF,78.21,clear sky,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
705,Hope,US,77.07,few clouds,33.6671,-93.5916,Best Western Hope
708,Hamada,JP,77.72,broken clouds,34.8833,132.0833,Shimane Hamada Washington Hotel Plaza
709,Kandrian,PG,82.60,moderate rain,-6.2167,149.5500,Kandrian Station
713,Darab,IR,78.78,clear sky,28.7519,54.5444,Naghshe shapoor Hotel


In [81]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [82]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</d1
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [64]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations=hotel_df[["Lat", "Lng"]]
fig=gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))